In [1]:
!tar xfvz /kaggle/input/ultralitycs/archive.tar.gz
!pip install --no-index --find-links=./packages ultralytics
!rm -rf ./packages

./packages/
./packages/nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl
./packages/triton-3.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
./packages/psutil-6.1.1-cp36-abi3-manylinux_2_12_x86_64.manylinux2010_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl
./packages/typing_extensions-4.12.2-py3-none-any.whl
./packages/nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl
./packages/MarkupSafe-3.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
./packages/ultralytics_thop-2.0.14-py3-none-any.whl
./packages/nvidia_nccl_cu12-2.21.5-py3-none-manylinux2014_x86_64.whl
./packages/scipy-1.15.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
./packages/networkx-3.4.2-py3-none-any.whl
./packages/torchvision-0.21.0-cp310-cp310-manylinux1_x86_64.whl
./packages/nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl
./packages/nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl
./packages/charset_normalizer-3.4.

In [2]:
!cp -r '/kaggle/input/hengck-czii-cryo-et-01/wheel_file' '/kaggle/working/'
!pip install /kaggle/working/wheel_file/asciitree-0.3.3/asciitree-0.3.3
!pip install --no-index --find-links=/kaggle/working/wheel_file zarr

Processing ./wheel_file/asciitree-0.3.3/asciitree-0.3.3
  Preparing metadata (setup.py) ... done
  Created wheel for asciitree: filename=asciitree-0.3.3-py3-none-any.whl size=5034 sha256=1e3b535bc6e32529916bec8da62b6f66033e5f70e8d7b6e4cf3f547a90d9d242
  Stored in directory: /root/.cache/pip/wheels/72/5f/18/c0251c9cff85c62eda5978bec3fd2e2bf68c30d9b89d523146
Successfully built asciitree
Looking in links: /kaggle/working/wheel_file
Processing ./wheel_file/zarr-2.18.3-py3-none-any.whl
Processing ./wheel_file/numcodecs-0.13.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (from zarr)
Processing ./wheel_file/fasteners-0.19-py3-none-any.whl (from zarr)


In [3]:
import zarr
from ultralytics import YOLO
from tqdm import tqdm
import glob, os
import torch

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

In [5]:
import sys
sys.setrecursionlimit(10000)

In [6]:
import warnings
warnings.simplefilter('ignore')
np.warnings = warnings

In [7]:
model = YOLO("/kaggle/input/czii-yolo11-training-baseline/runs/detect/train/weights/best.pt")

In [8]:
runs = sorted(glob.glob('/kaggle/input/czii-cryo-et-object-identification/test/static/ExperimentRuns/*'))
runs = [os.path.basename(x) for x in runs]
runs[:5]

['TS_5_4', 'TS_69_2', 'TS_6_4']

In [9]:
particle_names = ['apo-ferritin', 'beta-amylase', 'beta-galactosidase', 'ribosome', 'thyroglobulin', 'virus-like-particle']

In [10]:
p2i_dict = {
        'apo-ferritin': 0,
        'beta-amylase': 1,
        'beta-galactosidase': 2,
        'ribosome': 3,
        'thyroglobulin': 4,
        'virus-like-particle': 5
    }

i2p = {v:k for k, v in p2i_dict.items()}

In [11]:
particle_radius = {
        'apo-ferritin': 60,
        'beta-amylase': 65,
        'beta-galactosidase': 90,
        'ribosome': 150,
        'thyroglobulin': 130,
        'virus-like-particle': 135,
    }

In [12]:
class PredAggForYOLO:
    def __init__(self, first_conf=0.2, final_conf=0.3, conf_coef=0.75):
        self.first_conf = first_conf # threshold of confidence yolo
        self.final_conf = final_conf # final threshold score (not be used in version 14)
        self.conf_coef = conf_coef # if found many points, give bonus
        self.particle_confs = [0.5, 0.0, 0.2, 0.5, 0.2, 0.5] # be strict to easy labels 

    def convert_to_8bit(self, x):
        lower, upper = np.percentile(x, (0.5, 99.5))
        x = np.clip(x, lower, upper)
        x = (x - x.min()) / (x.max() - x.min() + 1e-12) * 255
        return x.round().astype("uint8")

    # depth first search.
    # aggregate the coordinates and confidence scores of connected graphs.
    def dfs(self, v):
        self.passed[v] = True
        self.conf_sum += self.pdf.iloc[v].confidence
        self.cx += self.pdf.iloc[v].x
        self.cy += self.pdf.iloc[v].y
        self.cz += self.pdf.iloc[v].z
        self.nv += 1
        for next_v in self.adjacency_list[v]:
            if (self.passed[next_v]): continue
            self.dfs(next_v)

    # main routine.
    def make_predict_yolo(self, r, model):
        vol = zarr.open(f'/kaggle/input/czii-cryo-et-object-identification/test/static/ExperimentRuns/{r}/VoxelSpacing10.000/denoised.zarr', mode='r')
        vol = vol[0]
        vol2 = self.convert_to_8bit(vol)
        n_imgs = vol2.shape[0]
    
        df = pd.DataFrame()
    
        pts = []
        confs = []
        xs = []
        ys = []
        zs = []
        
        for i in range(n_imgs):
            # Unfortunately the image size needs to be a multiple of 32.
            tmp_img = np.zeros((630, 630))
            tmp_img[:] = vol2[i]
    
            inp_arr = np.stack([tmp_img]*3,axis=-1)
            inp_arr = cv2.resize(inp_arr, (640,640))
            res = model.predict(inp_arr, save=False, imgsz=640, conf=self.first_conf, device="0", batch=1, verbose=False)
            for j, result in enumerate(res):
                boxes = result.boxes # Boxes object for bounding box outputs    
                for k in range(len(boxes.cls)):
                    ptype = i2p[boxes.cls.cpu().numpy()[k]] # particle type
                    conf = boxes.conf.cpu().numpy()[k] # confidence score
                    # YOLO can infer (start_x, end_x, start_y, end_y)
                    xc = (boxes.xyxy[k,0] + boxes.xyxy[k,2]) / 2.0 * 10 * (63/64)
                    yc = (boxes.xyxy[k,1] + boxes.xyxy[k,3]) / 2.0 * 10 * (63/64)
                    zc = i * 10 + 5
    
                    pts.append(ptype)
                    confs.append(conf)
                    xs.append(xc.cpu().numpy())
                    ys.append(yc.cpu().numpy())
                    zs.append(zc)           
                
        df['particle_type'] = pts
        df['confidence'] = confs
        df['x'] = xs
        df['y'] = ys
        df['z'] = zs

        # df includes overall canditate of CIRCLE. 
        df = df.sort_values(['particle_type', 'z'], ascending=[True, True])
    
        agg_df = []

        # infer center of sphere each particle types
        for pidx, p in enumerate(particle_names):
            if p == 'beta-amylase':
                continue
            pdf = df[df['particle_type']==p].reset_index(drop=True)
            self.pdf = pdf
            p_rad = particle_radius[p]

            # The distance between the x and y coordinates of adjacent slices is expected to be very small.
            xy_tol = p_rad / 16.0
            xy_tol_p2 = xy_tol ** 2

            # define the graph
            self.adjacency_list = [[] for _ in range(len(pdf))]
            # which already passed in dfs
            self.passed = [False for _ in range(len(pdf))]

            # Connect two points when they are close enough
            for i in range(len(pdf)):
                x1 = pdf['x'].iloc[i]
                y1 = pdf['y'].iloc[i]
                z1 = pdf['z'].iloc[i]
                for j in range(i+1, len(pdf), 1):
                    x2 = pdf['x'].iloc[j]
                    y2 = pdf['y'].iloc[j]
                    z2 = pdf['z'].iloc[j]
                    # Can be pruned. thanks to min fuka (@minfuka)
                    if abs(z1-z2)>20:
                        break
    
                    dist_p2 = (x1-x2)**2 + (y1-y2)**2
                    if dist_p2<xy_tol_p2 and dist_p2+(z1-z2)**2 < p_rad**2 and abs(z1-z2)<=20:
                        self.adjacency_list[i].append(j)
                        self.adjacency_list[j].append(i)

            rdf = pd.DataFrame()
            cxs = []
            cys = []
            czs = []

            # Perform DFS on all points and find the center of the sphere from the average of the coordinates
            for i in range(len(pdf)):
                self.conf_sum = 0
                self.nv = 0
                self.cx = 0
                self.cy = 0
                self.cz = 0
                if not self.passed[i]:
                    self.dfs(i)

                # Different confidence for different particle types
                if self.nv>=2 and self.conf_sum / (self.nv**self.conf_coef) > self.particle_confs[pidx]:
                    cxs.append(self.cx / self.nv)
                    cys.append(self.cy / self.nv)
                    czs.append(self.cz / self.nv)

            rdf['experiment'] = [r] * len(cxs)
            rdf['particle_type'] = [p] * len(cys)
            rdf['x'] = cxs
            rdf['y'] = cys
            rdf['z'] = czs

            agg_df.append(rdf)

       
        return pd.concat(agg_df, axis=0)

In [13]:
agent = PredAggForYOLO(first_conf=0.15, final_conf=0.2, conf_coef=0.5)

In [14]:
subs = []

In [15]:
import time

In [16]:
%%time
tick = time.time()
for r in tqdm(runs, total=len(runs)):
    df = agent.make_predict_yolo(r, model)
    subs.append(df)
tock = time.time()

100%|██████████| 3/3 [00:57<00:00, 19.31s/it]

CPU times: user 47.5 s, sys: 6.42 s, total: 53.9 s
Wall time: 57.9 s


In [17]:
print(f'estimated predict time is {(tock-tick)/3*500:.4f} seconds')

estimated predict time is 9653.7811 seconds


In [18]:
submission = pd.concat(subs).reset_index(drop=True)
submission.insert(0, 'id', range(len(submission)))

In [19]:
submission.to_csv("submission.csv", index=False)
submission.head()

,id,experiment,particle_type,x,y,z
0,0,TS_5_4,apo-ferritin,5878.604492,5129.602600,90.0
1,1,TS_5_4,apo-ferritin,5745.687402,5111.375684,105.0
2,2,TS_5_4,apo-ferritin,5712.691685,4993.255999,115.0
3,3,TS_5_4,apo-ferritin,5299.776074,4167.666406,145.0
4,4,TS_5_4,apo-ferritin,2806.420441,1636.127884,210.0
